In [ ]:
import requests
import pandas as pd
from ratelimit import limits

In [ ]:
base_url = 'https://api.opendota.com/api/'
noone_id = '106573901'
actions = {
    '1': 'move_to_position',
    '2': 'move_to_target',
    '3': 'attack_position',
    '4': 'attack_target',
    '10': 'hold_position'
}

In [ ]:
params = {'limit': 3, 'game_mode': 2}
r = requests.get('{base_url}/players/{account_id}/matches'.format(base_url=base_url,account_id=noone_id), params=params)

In [70]:
df = pd.DataFrame.from_dict(r.json(), orient='columns')

In [ ]:
df.head()

In [ ]:
@limits(calls=3, period=1)
def get_match(match_id):
    r = requests.get('{base_url}/matches/{match_id}'.format(base_url=base_url,match_id=match_id))
    return r.json()

In [ ]:
for index, row in df.iterrows():
    data = get_match(row['match_id'])
    player = list(filter(lambda p: p['player_slot'] == row['player_slot'], data['players']))
    player = player[0]
    total = 0
    for action in player['actions']:
        total = total + player['actions'][action]
        if action not in actions:
            continue
        df.at[index, actions[action]] = player['actions'][action]
    df.at[index, 'actions_per_minute'] = total / row['duration'] * 60

In [69]:
df.head()

,assists,deaths,duration,game_mode,hero_id,kills,leaver_status,lobby_type,match_id,party_size,...,radiant_win,skill,start_time,version,move_to_position,move_to_target,attack_position,attack_target,hold_position,actions_per_minute
0,4,6,1347,2,114,3,0,1,4862724858,10,...,False,None,1561406779,21,8741,249,20,1143,380,499.554566
1,18,1,2482,2,106,11,0,1,4862639921,10,...,True,None,1561402487,21,17523,384,46,2136,485,529.580983
2,0,2,1422,2,49,1,0,1,4862569782,10,...,False,None,1561399019,21,9036,217,21,1682,370,510.210970
3,13,0,1859,2,36,5,0,1,4861754075,10,...,True,None,1561374092,21,11904,310,53,2051,277,491.845078
4,15,5,3167,2,80,2,0,1,4861607434,10,...,False,None,1561368799,21,18162,604,71,3294,576,454.215346
